In [ ]:
import pandas as pd
import re
import requests

from time import sleep
from bs4 import BeautifulSoup

In [ ]:
# Extraindo dados do primeiro forum escolhido

def extract_text(text, pattern):
    try:
        found = re.search(pattern, text).group(1)
    except AttributeError:
        found = '0'
    return found

def extract_post(link):
    url  = "https://forumjuridico.org/" + link
    page = requests.get(url, headers={"User-Agent": "XY"})
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find("blockquote", class_="messageText ugc baseHtml").text
    #sleep(1.0)
    return results

# Bloco onde definiremos a quantidade de paginas que vamos buscar dentro do fórum (cada pagina possui 20 perguntas)
# e extraímos as informacoes mais basicas de cada uma (titulo, data, # de visitas, # de respostas e link para a postagem).

areas = ['direito-do-trabalho.17', 'direito-de-familia.11', 'direito-administrativo.15',
        'direito-civil-empresarial-e-do-consumidor.10', 'direito-penal-e-processo-penal.12']

title_lst = []
date_lst = []
answer_lst = []
visit_lst = []
url_lst = []
post_lst = []
area_lst = []
page_old = None

for area in areas:
    
    for i in range(0, 100):
        
        url  = "https://forumjuridico.org/forums/" + area +"/page-" + str(i) + "?_params=Array"
        page = requests.get(url, headers={"User-Agent": "XY"})
             
        # Sair do loop caso cheguemos na ultima pagina de uma determinada area no forum.
        # Podemos fazer isso pois ao chegar na ultima pagina, se aumentaros o valor de i 
        # a ultima pagina continuará sendo mostrada
        
        if page == page_old:
            break
        
        # Request para extrair o documento html
        
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find(id="content")
        elements = soup.find_all("li", class_="discussionListItem visible")
        page_old = page
        
        # Extração das informações pertinentes dentro de cada pagina

        for element in elements:
            title_lst.append(element.find("h3", class_="title").text.strip())
            try:
                date_lst.append(element.find("span", class_="DateTime").text.strip())
            except AttributeError:
                # A pagina de direito de familia utiliza um atributo diferente para o div, entao tratamos o erro aqui
                # para buscar o atributo correto
                date_lst.append(element.find("abbr", class_="DateTime").text.strip())
                
            answer_lst.append(extract_text(element.find("dl", class_="major").text.strip(), '.+?(\d)'))
            visit_lst.append(extract_text(element.find("dl", class_="minor").text.strip(), '.+?(\d)'))
            url_lst.append(extract_text(str(element.find("h3", class_="title").find("a", class_="PreviewTooltip")), 
                                       'href=\"(.+?)\"\ title'))
            post_lst.append(extract_post(extract_text(str(element.find("h3", class_="title").find("a", class_="PreviewTooltip")), 
                                       'href=\"(.+?)\"\ title')))
            area_lst.append(area)

    # Caso seja a primeira area, criar o data frame. Caso contrario, realizar o append
    
    if area == 'direito-do-trabalho.17':
        posts_df = pd.DataFrame(
            {'title': title_lst,
             'date': date_lst,
             'answers': answer_lst,
             'visits': visit_lst,
             'url': url_lst,
             'post': post_lst,
             'area': area_lst
            })
    else:
        posts_df = posts_df.append(
            pd.DataFrame(
            {'title': title_lst,
             'date': date_lst,
             'answers': answer_lst,
             'visits': visit_lst,
             'url': url_lst,
             'post': post_lst,
             'area': area_lst
            })
        )

posts_df.head()

posts_df.to_csv('dados\posts.csv', sep='|', header=True, index=False)

In [ ]:
# Extraindo dados de do segundo forum escolhido

def extract_text(text, pattern):
    try:
        found = re.search(pattern, text).group(1)
    except AttributeError:
        found = 0
    return found

def extract_post(link):
    url  = "https://www.perguntedireito.com.br" + link
    page = requests.get(url, headers={"User-Agent": "XY"})
    soup = BeautifulSoup(page.content, "html.parser")
    results = extract_text(str(soup), 'itemprop\=\"text\"\>(.+?)\<\/div')
    #sleep(1.0)
    return results

# Bloco onde definiremos a quantidade de paginas que vamos buscar dentro do fórum (cada pagina possui 10 perguntas)
# e extraímos as informacoes mais basicas de cada uma (titulo, data, # de visitas, # de respostas e link para a postagem).

for i in range(0, 20):

    url  = "https://www.perguntedireito.com.br/questions/direito-empresarial?start=" + str(i * 10)
    page = requests.get(url, headers={"User-Agent": "XY"})

    # Request para extrair o documento html

    soup = BeautifulSoup(page.content, "html.parser")
    elements = soup.find_all("div", class_="qa-q-list-item")

    title_lst = []
    date_lst = []
    answer_lst = []
    url_lst = []
    post_lst = []

    for element in elements:
        title_lst.append(element.find("div", class_="qa-q-item-title").text.strip())
        date_lst.append(element.find("span", class_="qa-q-item-when-data").text.strip())
        answer_lst.append(extract_text(element.find("span", class_="qa-a-count-data").text.strip(), '.+?(\d)'))
        url_lst.append(extract_text(str(element.find("div", class_="qa-q-item-title")), 
                                   'href=\"(.+?)\"\>'))
        post_lst.append(
            extract_post(
                extract_text(
                    str(
                        element.find("div", class_="qa-q-item-title")
                    ),'href=\"(.+?)\"\>')
            )
        )

    # Transformando as listas em um dataframe e visualizando o resultado

    if i == 0:
        empresarial_df = pd.DataFrame(
            {'title': title_lst,
             'date': date_lst,
             'answers': answer_lst,
             'url': url_lst,
             'post': post_lst,
             'area': 'empresarial'
            })
    else:
        empresarial_df = empresarial_df.append(
            pd.DataFrame(
            {'title': title_lst,
             'date': date_lst,
             'answers': answer_lst,
             'url': url_lst,
             'post': post_lst,
             'area': 'empresarial'
            })
        )

empresarial_df.head()

empresarial_df.to_csv('dados\posts_2.csv', sep='|', header=True, index=False)

In [ ]:
# Extraindo dados do terceiro forum escolhido

def extract_text(text, pattern):
    try:
        found = re.search(pattern, text).group(1)
    except AttributeError:
        found = '0'
    return found

def extract_post(link):
    page = requests.get(link, headers={"User-Agent": "XY"})
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find("div", class_="question-body ask_content").text
    return results

url_base = 'https://jus.com.br/duvidas/'
areas = [
    'direito-administrativo/',
    'direito-do-trabalho/',
    'direito-penal/',
    'direito-processual-penal/',
    'direito-civil/',
    'direito-do-consumidor/',
    'direito-de-familia/'
]

title_lst = []
date_lst = []
answer_lst = []
like_lst = []
url_lst = []
post_lst = []
area_lst = []
page_old = None


for area in areas:
    
    for i in range(0, 100):
        
        url  = url_base + area + '/p/' + str(i+1)
        page = requests.get(url, headers={"User-Agent": "XY"})
             
        # Sair do loop caso cheguemos na ultima pagina de uma determinada area no forum.
        # Podemos fazer isso pois ao chegar na ultima pagina, se aumentaros o valor de i 
        # a ultima pagina continuará sendo mostrada

        if page == page_old:
            break
        
        # Request para extrair o documento html
        
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find(id="questions")
        elements = results.find_all("li", class_="")
        page_old = page
        
        # Extração das informações pertinentes dentro de cada pagina

        for element in elements:

            if len(str(element)) < 400:
                continue

            title_lst.append(element.find("a", class_="title").text.strip())
            date_lst.append(element.find("abbr", class_="timeago").text.strip())
            try:
                answer_lst.append(element.find("span", class_="answers-count striped").text.strip())
            except Exception as e:
                answer_lst.append("0")
            like_lst.append(element.find("i", class_="fa fa-heart").text.strip())
            # <span class="likes"><i class="fa fa-heart"></i> 0</span>
            url_lst.append(extract_text(str(element.find("a", class_="title")), 
                                       'href=\"(.+?)\"\>'))
            post_lst.append(extract_post(extract_text(str(element.find("a", class_="title")), 
                                       'href=\"(.+?)\"\>')))
            area_lst.append(area)

    # Caso seja a primeira area, criar o data frame. Caso contrario, realizar o append
    
    if area == 'direito-administrativo/':
        posts_df = pd.DataFrame(
            {'title': title_lst,
             'date': date_lst,
             'answers': answer_lst,
             'likes': like_lst,
             'url': url_lst,
             'post': post_lst,
             'area': area_lst
            })
    else:
        posts_df = posts_df.append(
            pd.DataFrame(
            {'title': title_lst,
             'date': date_lst,
             'answers': answer_lst,
             'likes': like_lst,
             'url': url_lst,
             'post': post_lst,
             'area': area_lst
            })
        )


posts_df.to_csv('dados\posts_4.csv', sep='|', header=True, index=False)
